In [11]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
import matplotlib.pyplot as plt
import json
import scipy.stats as stats
import math

import category_encoders as ce

import statsmodels.api as sm
import statsmodels.stats as sm_stats
import statsmodels.stats.api as sms

import vizualizacia_funkcie as visual

from sklearn.experimental import enable_iterative_imputer 
from sklearn import impute 
from sklearn import preprocessing
from sklearn import pipeline
from sklearn import base
from sklearn import compose
from datetime import datetime
from datetime import date

import imblearn

In [2]:
df1 = pd.read_csv("./data/personal_train.csv", index_col=0)
df2 = pd.read_csv("./data/other_train.csv", index_col=0)

data = df1.drop(columns=["address"]).set_index("name").join(df2.set_index("name"), how="right").reset_index()
data

,name,age,sex,date_of_birth,address,race,marital-status,occupation,pregnant,education-num,...,education,fnlwgt,class,std_glucose,income,medical_info,native-country,hours-per-week,capital-loss,workclass
0,Aaron Davis,56,Female,1962-12-27,"379 Rodriguez Inlet\r\nFisherbury, MD 83114",White,Divorced,Other-service,f,3.0,...,5th-6th,199298.0,0.0,41.806596,<=50K,"{'mean_oxygen':'1.647157191','std_oxygen':'14....",?,45.0,0.0,Local-gov
1,Aaron Dini,50,Male,1969-05-22,"105 Medina Springs Suite 462\r\nEmilyland, KS ...",White,Divorced,Sales,f,10.0,...,Some-college,120939.0,0.0,53.615229,<=50K,"{'mean_oxygen':'1.380434783','std_oxygen':'15....",United-States,40.0,0.0,Self-emp-not-inc
2,Aaron Gann,25,Male,1994-07-26,59685 Thomas Extension Apt. 142\r\nFieldsborou...,Black,Never-married,?,f,4.0,...,7th-8th,107112.0,0.0,50.476073,<=50K,"{'mean_oxygen':'78.30183946','std_oxygen':'79....",United-States,30.0,0.0,?
3,Aaron Gregg,42,Female,1977-04-16,"10883 Williams Locks Suite 821\r\nEast Carmen,...",Black,Widowed,Transport-moving,f,9.0,...,HS-grad,157942.0,0.0,35.976649,<=50K,"{'mean_oxygen':'1.489130435','std_oxygen':'10....",United-States,40.0,0.0,Local-gov
4,Aaron Lindley,-4900,Female,1969-12-11 00:00:00,"564 Stevenson Park Apt. 839\r\nRodriguezville,...",White,Never-married,Sales,F,9.0,...,HS-grad,156084.0,0.0,54.280153,<=50K,"{'mean_oxygen':'1.194816054','std_oxygen':'11....",United-States,40.0,0.0,Private
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3978,Young Provines,51,Male,1968-05-19,"7993 Sharon Roads Suite 311\r\nPort Daniel, GA...",White,Married-civ-spouse,Exec_managerial,FALSE,11.0,...,Assoc-voc,83444.0,1.0,44.303479,>50K,"{'mean_oxygen':'3.16722408','std_oxygen':'22.3...",United-States,43.0,0.0,Private
3979,Zachary Grandfield,-5800,Male,1960/12/18,"77606 Maxwell Rest Apt. 485\r\nPort Jennifer, ...",Black,Married-civ-spouse,Transport-moving,f,10.0,...,Some-college,343849.0,1.0,29.456767,<=50K,"{'mean_oxygen':'2.994147157','std_oxygen':'21....",United-States,40.0,0.0,Private
3980,Zachary Miller,46,Male,1973-08-07,"96654 Marc Stream\r\nEast Cassandra, VT 84597",White,Married-civ-spouse,Adm-clerical,f,13.0,...,Bachelors,162327.0,0.0,43.191434,>50K,"{'mean_oxygen':'1.62541806','std_oxygen':'14.0...",?,50.0,1902.0,Private
3981,Zachary Seltzer,64,Female,1955-06-08 00 00 00,"38371 Crystal Stravenue\r\nPerkinsmouth, DE 20764",White,Divorced,Sales,f,7.0,...,11th,444089.0,1.0,34.132553,<=50K,"{'mean_oxygen':'66.53260870000001','std_oxygen...",United-States,40.0,0.0,Private


In [3]:
columns = list(data.columns.values)
columns

['name',
 'age',
 'sex',
 'date_of_birth',
 'address',
 'race',
 'marital-status',
 'occupation',
 'pregnant',
 'education-num',
 'relationship',
 'skewness_glucose',
 'mean_glucose',
 'capital-gain',
 'kurtosis_glucose',
 'education',
 'fnlwgt',
 'class',
 'std_glucose',
 'income',
 'medical_info',
 'native-country',
 'hours-per-week',
 'capital-loss',
 'workclass']

### Operacie

- PIP -pipeline
- FT - FuntctionTransformer
- CT - ColumnTransformer
- EST - estimator
- MyT - CustomTransformer
- MPH - mensi pocet hodnot

### Preprocessing kroky

- 'name', - DELETE

- 'age', - (1) DELETE, pouzit datum na imputaciu (MyT), SimpleImputer, transformacia, outliery, scaling
- 'age', - (2) nastavenie nan pre zle hodnoty, MyT, SimpleImputer, transformacia, outliery, scaling

- 'sex', - SimpleImputer, OneHot
- 'date_of_birth', - MyT (pouziva sa pri agi), DELETE
- 'race', - DELETE

- 'address', - zistenie mesta/statu z adresy, DELETE
- 'city' - ???
- 'state' - ???

- 'marital-status', - MPH, imputacia marital-status + relationship (MyT) asi KNN, SimpleImputer, OneHot
- 'relationship', - MPH, imputacia ..., SimpleImputer, OneHot

- 'occupation', - MPH, imputacia (skupina occupation, workclass, education, hours-per-week, income) asi KNN, SimpleImputer, OneHot 
- 'pregnant', - DELETE
- 'education-num', - MyT pre education, DELETE
- 'education', - MyT pre education, MPH, imputacia ..., OrdinalEncoder
- 'workclass' - MPH, imputacia..., OneHot
- 'income', - imputacia..., OneHot
- 'hours-per-week', - diskretizacia, imputacia...., OrdinalEncoder
 
- 'skewness_glucose', - - transformacia + outlier removal, imputacia oxygenov+glukoz, scaling
- 'mean_glucose', - - transformacia + outlier removal, imputacia oxygenov+glukoz, scaling
- 'std_glucose', - - transformacia + outlier removal, imputacia oxygenov+glukoz, scaling
- 'kurtosis_glucose', - - transformacia + outlier removal, imputacia oxygenov+glukoz, scaling

- 'skewness_oxygen', - transformacia + outlier removal, imputacia oxygenov+glukoz, scaling
- 'mean_oxygen', - transformacia + outlier removal, imputacia oxygenov+glukoz, scaling
- 'std_oxygen', - transformacia + outlier removal, imputacia oxygenov+glukoz, scaling
- 'kurtosis_oxygen', - transformacia + outlier removal, imputacia oxygenov+glukoz, scaling


- 'capital-gain', - DELETE
- 'capital-loss', - DELETE

- 'fnlwgt', - DELETE
- 'class', - remove missing values

- 'medical_info', - parsovanie, DELETE
- 'native-country', - DELETE

### Po encodingoch by to chcelo prejst atributy cez VarianceThreshold

### Preprocessing kroky

- 'age', - (1) DELETE, pouzit datum na imputaciu (MyT), SimpleImputer, transformacia + outliery, 
- 'age', - (2) nastavenie nan pre zle hodnoty, MyT, SimpleImputer, transformacia + outliery, 

- 'sex', - SimpleImputer,
- 'date_of_birth', - MyT (pouziva sa pri agi), DELETE

- 'address', - zistenie mesta/statu z adresy, DELETE
- 'city' - ???
- 'state' - ???

- 'marital-status', - MPH, imputacia marital-status + relationship (MyT) asi KNN, SimpleImputer, 
- 'relationship', - MPH, imputacia ..., SimpleImputer, 

- 'occupation', - MPH, imputacia (skupina occupation, workclass, education, hours-per-week, income) asi KNN, SimpleImputer,
- 'education-num', - MyT pre education, DELETE
- 'education', - MyT pre education, MPH, imputacia ..., 
- 'income', - imputacia...,
- 'hours-per-week', - diskretizacia, imputacia...., 
 
- 'skewness_glucose', - - transformacia + outlier removal, 
- 'mean_glucose', - - transformacia + outlier removal, 
- 'std_glucose', - - transformacia + outlier removal, 
- 'kurtosis_glucose', - - transformacia + outlier removal, 

- 'skewness_oxygen', - transformacia + outlier removal, 
- 'mean_oxygen', - transformacia + outlier removal, 
- 'std_oxygen', - transformacia + outlier removal, 
- 'kurtosis_oxygen', - transformacia + outlier removal,

- 'class', - remove missing values

### Po encodingoch by to chcelo prejst atributy cez VarianceThreshold

# Scikit Pipeline

In [4]:
#funckia, ktora mergne zaznamy, ktore su rovnake
def piece_datarows_together(data):
    
    data = data.copy().set_index("name")
    
    #toto nam vrati dataset, ktory obsahuje vsetky duplikaty, s ktorymi budeme pracovat
    #proste to vrati data, ktore maju index, ktory je v datasete viac ako raz pouzity
    duplicated = data[data.index.duplicated(keep=False)]
    
    index_values = duplicated.index.unique()
    
    #najprv vsetky hodnoty prenesieme do prveho vyskytu zaznamu daneho pacienta v datasete
    for idx in index_values:
        mini_dataset = duplicated.loc[idx] #toto vrati viacero zaznamov s rovnakych idx
        
        #zistim si, ktore atributy su nullove pre presne prvy zaznam a pre konkretne nullove atributy budem nadalej hladat
        #nenullovu hodnotu v ostatnych zaznamoch s rovnakym idx
        missing_mask = mini_dataset.iloc[0].isnull()
        attributes = mini_dataset.columns.values
        missing_attributes = attributes[missing_mask]
        
        #tu replacujem null hodnoty za nenullove
        for attr in missing_attributes:
            not_null = mini_dataset[attr][mini_dataset[attr].notnull()]
            
            if len(not_null) != 0:
                mini_dataset.iloc[0][attr] = not_null.values[0]
        
        
    #teraz uz mozme vymazat vsetky druhe, resp. ostatne zaznamy pacienta
    duplicated_mask = data.index.duplicated(keep="first")
    
    data = data.reset_index()
    duplicated_indices = data.index.values[duplicated_mask]
    
    
    return data.drop(index=duplicated_indices).reset_index(drop=True)


In [5]:
def one_proper_df(df1, df2, return_X_y=True):
    data = df1.drop(columns=["address"]).set_index("name").join(df2.set_index("name"), how="right").reset_index()
    data = piece_datarows_together(data)
    
    if return_X_y == True:
        X = data.drop(columns=["class"])
        y = data["class"]
        return X,y
    
    else:
        return data 
    

In [68]:
def remove_useless_features(X):
    useless_cols = ["name", "race", "pregnant", "capital-loss", "capital-gain", "fnlwgt", "native-country"]
    return X.drop(columns=useless_cols)

def add_oxygen_features(X):
    return X.apply(get_oxygen_stats, axis=1)
    
def get_oxygen_stats(row):
    
    string = row["medical_info"]
    
    if string is np.nan:
        return row
    
    string = string.replace("\'", "\"")
    di = json.loads(string)
    
    for k in di.keys():
        row[k] = float(di[k])
        
    return row

In [7]:
prvy_pip = pipeline.Pipeline(steps=[
    ("feature_removal", preprocessing.FunctionTransformer(remove_useless_features)),
    ("add_oxygen_attr", preprocessing.FunctionTransformer(remove_useless_features)),
    
])

# Imputing

In [ ]:
class CustomCatImputing(base.BaseEstimator, base.TransformerMixin):
    
    def __init__(self, imputer_type="knn"):
        self.ordinal_encoder = None
        self.imputer = None
        self.imputer_type = impute_type
        
    def fit(self, X, y=None):
        
        print("FIT")
        
        columns = X.columns.values

        null_values = pd.DataFrame(index=pd.Index([-1]), columns=columns, data=[[np.nan for i in range(len(columns))]])
        X = pd.concat([null_values,X])

        self.ordinal_encoder = ce.ordinal.OrdinalEncoder(handle_missing="return_nan", handle_unknown="return_nan")
        X = self.ordinal_encoder.fit_transform(X)
        
        X = X[1:]
        
        if self.imputer_type == "knn":
            self.imputer = impute.KNNImputer()
            X = self.imputer.fit(X)
        
        elif self.imputer_type == "iterative":

            self.imputer = impute.IterativeImputer(max_iter=20, random_state=42, initial_strategy="most_frequent", 
                                                  min_value=X.min(), max_value=X.max())


            try:
                X = self.imputer.fit(X)
            except (ValueError, np.linalg.LinAlgError):
                print("Jeden error bol trapnuty, kedy funkcii vadili NaNs. Tento error je ale divny, lebo mu to vadi", \
                  "len prvy krat, a potom to uz ide...")
                X = self.imputer.fit(X)
            
        return self
               

    def transform(self, X):
        print("TRANSFORM")
        columns = X.columns.values
    
        X = self.ordinal_encoder.transform(X)
        X = self.imputer.transform(X).round()
        
        X = pd.DataFrame(data=X, columns=columns)
        
        X = self.ordinal_encoder.inverse_transform(X)
        
        return X
    

In [ ]:
medical_info_impute = pipeline.Pipeline(steps=[
    ("oxygen_n_glucose_impute", impute.IterativeImputer(max_iter=20))
])

vztahy_attr_impute = pipeline.Pipeline(steps=[
    ("vztahy_impute", CustomCatImputing(imputer_type="knn")),
    ("simple_impute", impute.SimpleImputer(strategy="most_frequent"))
])

work_attr_impute = pipeline.Pipeline(steps=[
    ("work_impute", CustomCatImputing(imputer_type="knn")),
    ("simple_impute", impute.SimpleImputer(strategy="most_frequent"))
])

vztahy_attr = ["relationship", "marital-status"]
work_attr = ["workclass, ""]

impute_col_transf = compose.ColumnTransform(transformers=[
    ("oxygen_n_glucose_impute", medical_info_impute, oxygen_attr + glucose_attr),
    ("vztahy_impute", vztahy_attr_impute, oxygen_attr + glucose_attr),
    ("work_impute", work_attr_impute, oxygen_attr + glucose_attr),
])

# Scaling a Encoding

In [ ]:
scaling = pipeline.Pipeline(steps=[
    ("standard_scaler", preprocessing.StandardScaler())
])

onehot = pipeline.Pipeline(steps=[
    ("one_hot_enc", preprocessing.OneHotEncoding(handle_unknown="ignore"))
])

ord_mapping = [
    {"col": "education", "mapping": {
        "Attending-school": 1, 
        "HS-grad": 2
        "Edu after HS, no uni": 3,
        "Some-college": 4,
        "Bachelors": 5,
        "Masters/Doctorate": 6}},
    
    {"col": "hours-per-week", "mapping": {
        "<=35": 1,
        "35< hours <=45": 2,
        ">45": 3}}
]

ordinal = pipeline.Pipeline(steps=[
    ("ordinal_enc", preprocessing.OrdicalEncoding(mapping=ord_mapping, handle_unknown="return_nan")),
    ("impute_unknown", impute.SimpleImpute(strategy="most_frequent"))
])


In [ ]:
oxygen_attr = ["mean_oxygen", "std_oxygen", "kurtosis_oxygen", "skewness_oxygen"]
glucose_attr = ["mean_glucose", "std_glucose", "kurtosis_glucose", "skewness_glucose"]

scaling_attr = ["age"] + oxygen_attr + glucose_attr
onehot_attr = ["sex", "date_of_birth", "marital-status", "occupation", "workclass", "income"]
ordinal_attr = ["education", "hours-per-week"]

last_col_transf = compose.ColumnTransformer(transformers=[
    ("num_attr_scaling", scaling, scaling_attr),
    ("cat_attr_onehot_enc", onehot, onehot_attr),
    ("cat_attr_ordinal_enc", ordinal, ordinal_attr)
])

In [ ]:
posledny_pip = pipeline.Pipeline(steps=[
    ("scaling_n_encoding", last_col_transf)
])

In [72]:
X,y = one_proper_df(df1, df2, return_X_y=True)
#pip.fit_transform(X, y)

<ipython-input-4-7f4a873f9dd0>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_dataset.iloc[0][attr] = not_null.values[0]


In [103]:
class MyResampler(base.BaseEstimator):
    
    def fit_resample(self, X, y):
        print("fit_resample")
        return self.resample(X, y)
        
    def resample(self, X, y):
        print("RESAMPLE")
        
        print(X.columns)
        return X, y
        

In [111]:
pip = pipeline.Pipeline(steps=[
    ("skuska", MyResampler())
])

TypeError: Last step of Pipeline should implement fit or be the string 'passthrough'. 'MyResampler()' (type <class '__main__.MyResampler'>) doesn't

In [112]:
from sklearn import svm

pip = imblearn.pipeline.Pipeline(steps=[
    ("skuska", MyResampler()),
    ("x", "passthrough")
])